# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as `customer_churn.csv`. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company

Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logistic-regression-exercise').getOrCreate()

In [3]:
data = spark.read.csv('../data/customer_churn.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()
data.describe().show()
data.show(5)

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+----------------

In [5]:
data = data.withColumn('Onboard_date', data['Onboard_date'].cast('timestamp'))


In [6]:
data.groupBy('Location').count().count(), data.groupBy('Company').count().count()

(900, 873)

In [7]:
data.select('Location', 'Company').limit(50).show(truncate=False)

+--------------------------------------------------------+----------------------------+
|Location                                                |Company                     |
+--------------------------------------------------------+----------------------------+
|10265 Elizabeth Mission Barkerburgh, AK 89518           |Harvey LLC                  |
|6157 Frank Gardens Suite 019 Carloshaven, RI 17756      |Wilson PLC                  |
|1331 Keith Court Alyssahaven, DE 90114                  |Miller, Johnson and Wallace |
|13120 Daniel Mount Angelabury, WY 30645-4695            |Smith Inc                   |
|765 Tricia Row Karenshire, MH 71730                     |Love-Jones                  |
|6187 Olson Mountains East Vincentborough, PR 74359      |Kelly-Warren                |
|4846 Savannah Road West Justin, IA 87713-3460           |Reynolds-Sheppard           |
|25271 Roy Expressway Suite 147 Brownport, FM 59852-6150 |Singh-Cole                  |
|3725 Caroline Stravenue South C

In [8]:
from pyspark.sql.functions import substring_index, split, year, month

In [9]:
data\
    .withColumn('State', split(substring_index(data['Location'], ' ', -2), ' ').getItem(0))\
    .groupBy('State').count().show()

+-----+-----+
|State|count|
+-----+-----+
|   AZ|   14|
|   SC|   18|
|   LA|   17|
|   MN|   17|
|   AA|   44|
|   NJ|   13|
|   DC|   15|
|   OR|   10|
|   VA|   16|
|   RI|   14|
|   WY|    7|
|   KY|   17|
|   NH|   15|
|   MI|   15|
|   NV|   14|
|   WI|   15|
|   ID|   17|
|   CA|    9|
|   NE|   12|
|   CT|   12|
+-----+-----+
only showing top 20 rows



In [10]:
data\
    .withColumn('OnboardMonth', month(data['Onboard_date']))\
    .withColumn('OnboardYear', year(data['Onboard_date']))\
    .show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+------------+-----------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|OnboardMonth|OnboardYear|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+------------+-----------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|           8|       2013|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|           8|       2013|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|           6|       2016

In [11]:
from pyspark.ml.pipeline import Pipeline, Transformer
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer
from pyspark.ml.classification import LogisticRegression

In [12]:
class StateOnboardTransformer(Transformer):
    def __init__(self):
        super().__init__()

    def _transform(self, data):
        return data\
            .withColumn('State', split(substring_index(data['Location'], ' ', -2), ' ').getItem(0))\
            .withColumn('OnboardMonth', month(data['Onboard_date']))\
            .withColumn('OnboardYear', year(data['Onboard_date']))

In [13]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [14]:
state_onboard_transformer = StateOnboardTransformer()
state_indexer = StringIndexer(inputCol='State', outputCol='StateIndexed')
state_one_hot = OneHotEncoder(inputCol='StateIndexed', outputCol='StateOneHot')
assembler = VectorAssembler(
    inputCols=[
        'Age',
        'Total_Purchase',
        'Years',
        'Num_Sites',
        'OnboardMonth',
        'OnboardYear',
        'StateOneHot'
    ], outputCol='features'
)
lr = LogisticRegression(
    featuresCol='features', labelCol='Churn', predictionCol='prediction'
)

In [15]:
pp = Pipeline(stages=[
    state_onboard_transformer, state_indexer, state_one_hot, assembler, lr
])

In [16]:
train_data, test_data = data.randomSplit([.7, .3], seed=42)

In [17]:
model = pp.fit(train_data)

In [18]:
test_results = model.transform(test_data)

In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [20]:
test_results.select('Churn', 'rawPrediction', 'probability', 'prediction').show()

+-----+--------------------+--------------------+----------+
|Churn|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+----------+
|    0|[2.53090069774277...|[0.92627988157907...|       0.0|
|    0|[1.94084821564377...|[0.87444529927073...|       0.0|
|    0|[1.72914678839520...|[0.84930325259947...|       0.0|
|    1|[-0.5908072935345...|[0.35644964512785...|       1.0|
|    0|[2.71016663930531...|[0.93762389528490...|       0.0|
|    0|[6.37858497269313...|[0.99830535340231...|       0.0|
|    0|[3.44553786495546...|[0.96909779125975...|       0.0|
|    0|[3.65658442767678...|[0.97482936547913...|       0.0|
|    0|[0.81641289819105...|[0.69347436787323...|       0.0|
|    0|[5.70256805885729...|[0.99667371723735...|       0.0|
|    0|[5.11973750672742...|[0.99405792751683...|       0.0|
|    0|[3.21360801451285...|[0.96134317171075...|       0.0|
|    0|[8.96963152190320...|[0.99987280112212...|       0.0|
|    1|[2.32094720983515

In [21]:
evaluator = MulticlassClassificationEvaluator(labelCol='Churn', predictionCol='prediction')

for metric in ['accuracy', 'precisionByLabel', 'recallByLabel']:
    m = evaluator.setMetricName(metric).evaluate(test_results)
    print(f'{metric.upper()}: {m}')

ACCURACY: 0.8540772532188842
PRECISIONBYLABEL: 0.8811881188118812
RECALLBYLABEL: 0.9468085106382979


In [22]:
new_customers = spark.read.csv('../data/new_customers.csv', inferSchema=True, header=True)
new_customers.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [23]:
new_customers_predictions = model.transform(new_customers)

In [24]:
new_customers_predictions.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+-----+------------+-----------+------------+---------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|State|OnboardMonth|OnboardYear|StateIndexed|    StateOneHot|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+-----+------------+-----------+------------+---------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|   WI|           8|       2011|        10.0|(61,[10],[1.0])|(67,[0,1,2,3,4,5,...|[3.3395741